In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xtrack as xt
import xcoll as xc

In [2]:
def install_tidp(line, block_mvt=29e-3):
    tidp_ap_tot = 147e-3
    line.discard_tracker()
    tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
    line.collimators.install(names=['tidp.11434'], elements=[tidp])
    return tidp

def install_tcsm(line):
    tcsm = xc.EverestCollimator(length=1.83, gap=5, material=xc.materials.Carbon) # length is 1.83
    line.collimators.install(names=['tcsm.51932'], elements=[tcsm])
    return tcsm

def install_offmom_bpms_colls(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2):
    tw = line.twiss()
    tt = line.get_table()
    mask_disp = 5*np.sqrt(tw.betx*exn*pmass/nrj)+n_buckets*bucket_height*tw.dx > 0.025
    mask_bpm = ['bp' in name for name in tt.name]
    mask_aper = np.array(['aper' in name for name in tt.name])
    offmom_bpms = tt.name[mask_disp & mask_bpm & ~mask_aper]
    colls = []
    aper_to_remove = []
    for nn in offmom_bpms:
        aper_to_remove.append(f'{nn}.a_aper')
        aper_to_remove.append(f'{nn}.b_aper')
        if line[nn+'.a_aper'].__class__.__name__ == 'LimitEllipse':
            jaw = line[nn+'.a_aper'].a
        else:
            jaw = line[nn+'.a_aper'].max_x
        
        colls.append(xc.EverestCollimator(length=line[nn].length, material=xc.materials.Beryllium, jaw=jaw))
    line.remove(aper_to_remove)
    line.collimators.install(names=offmom_bpms, elements=colls)
    return colls

def remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2):
    "Remove apertures of off-momentum BPMs which give flanges as bottlenecks"
    tw = line.twiss()
    tt = line.get_table()
    mask_disp = 5*np.sqrt(tw.betx*exn*pmass/nrj)+n_buckets*bucket_height*tw.dx > 0.025
    mask_bpm = ['bp' in name for name in tt.name]
    mask_aper = np.array(['aper' in name for name in tt.name])
    offmom_bpms = tt.name[mask_disp & mask_bpm & ~mask_aper]
    aper_to_remove = [f'{name}{suffix}' for name in offmom_bpms for suffix in ('.a_aper', '.b_aper')]
    line.remove(aper_to_remove)

In [7]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
tt1 = line.get_table()
remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)
tt = line.get_table()
# tw = line.twiss()

#shift VEB apertures by 5.3 mm to account for the sagitta
veb_b_apers = tt.rows['veb.*.b_aper'].name
for name in veb_b_apers:
    line[name].shift_x += 5.3e-3

veb_a_apers = tt.rows['veb.*.a_aper'].name
for name in veb_a_apers:
    line[name].shift_x += 5.3e-3
#Houdt de a aperture steek?
tt = line.get_table()
tw = line.twiss()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [8]:
L1 = 34e-3
L2 = 70e-3

r1 = 41.5e-3
r2 = 60e-3-1.5e-3
r3 = 51.5e-3

changed = [10110, 11110, 11310, 12510, 13510, 20910, 21110, 22510, 23510, 30110, 30910,
32510, 33510, 40110, 40910, 41110, 42510, 42710, 51110, 52510, 53510, 60110,
61110, 62510, 63510]

In [9]:
# Add apertures to elements in flanges that do not have one yet
env = line.env
insertions = []
for qd_number in changed:
    flange_number = qd_number - 9
    
    aper_start = f'vcak.{flange_number}.a_aper'
    aper_end = f'vcak.{flange_number}.b_aper'

    for nn, ee in zip(tt.rows[aper_start:aper_end].name, tt.rows[aper_start:aper_end].element_type):
        if not ee.startswith('Limit') and not ee.startswith('Drift') and not ee.startswith('Marker'):
            env.elements[f'{nn}_aper_enter'] = line[aper_start].copy()
            env.elements[f'{nn}_aper_exit'] = line[aper_end].copy()
            insertions.append(env.place(f'{nn}_aper_enter', at=f'{nn}@start'))
            insertions.append(env.place(f'{nn}_aper_exit', at=f'{nn}@end'))
line.insert(insertions, s_tol=1e-6)

Slicing line:   0%|          | 0/32246 [00:00<?, ?it/s]

In [11]:
tt = line.get_table()
start_flange_apers = []
end_flange_apers = []
for qd_number in changed:
    flange_number = qd_number - 9
    
    aper_start = f'vcak.{flange_number}.a_aper'
    aper_end = f'vcak.{flange_number}.b_aper'

    for nn, ee in zip(tt.rows[f'{aper_start}>>1':aper_end].name, tt.rows[f'{aper_start}>>1':aper_end].element_type):
        if ee.startswith('Drift') or ee.startswith('Marker'):
            continue
        elif ee.startswith('Limit'):
            next_aper = nn
            break
        else:
            raise ValueError(f'Unexpected element type {ee} for element {nn}')
    end_flange_apers.append(next_aper)
    start_flange_apers.append(aper_start)

In [23]:
flange_number = 13501
aper_start = start_flange_apers[4]
aper_end = end_flange_apers[4]

slicing_strategies = []
if tt.rows[aper_end].s - tt.rows[aper_start].s < L1 + L2:
    aper_exit = f'{aper_end.split('_')[0]}_aper_exit'
    if tt.rows[aper_exit].s - tt.rows[aper_start].s < L1 + L2:
        ele_to_slice = tt.rows[f'{aper_end}>>1'].name
        slicing_strategies.append(xt.Strategy(slicing=xt.Teapot(5), name=ele_to_slice))
        print('Slice')
    else:
        print('No slice needed, but change exit aperture')

line.slice_thick_elements(slicing_strategies)
    #check if L1+L2 still in element or not, if not no need to slice, but entry and exit apertures of element need to be changed
    


No slice needed, but change exit aperture


Slicing line:   0%|          | 0/32258 [00:00<?, ?it/s]

ValueError: No slicing strategy found for the element qf.10010: Quadrupole(k1=0.0116, k1s=0, length=3.08, num_multipole_kicks=np.int64(0), _order=np.int64(5), inv_factorial_order=0.00833, knl=array([0., 0., 0., 0., 0., 0.]), ksl=array([0., 0., 0., 0., 0., 0.]), edge_entry_active=np.uint64(0), edge_exit_active=np.uint64(0), _model=np.int64(0), _integrator=np.int64(0), radiation_flag=np.int64(0), delta_taper=0, _sin_rot_s=-999, _cos_rot_s=-999, _shift_x=0, _shift_y=0, _shift_s=0, _rot_x_rad=0, _rot_y_rad=0, _rot_s_rad_no_frame=0, rot_shift_anchor=0, _internal_record_id=RecordIdentifier(buffer_id=np.int64(0), offset=np.int64(0))).

In [26]:
arr = tt.rows[aper_exit].s - tt.rows[aper_start].s

In [30]:
if tt.rows[aper_end].s - tt.rows[aper_start].s < L1 + L2:
    print('ok')

ok


In [27]:
arr[0]

np.float64(0.7791999999978998)

In [20]:
aper_end

'lod.13502_aper_enter'

In [21]:
aper_exit = f'{aper_end.split('_')[0]}_aper_exit'

In [22]:
aper_exit

'lod.13502_aper_exit'

# Defining pipes as collimators to install and slice

Function gives definition of a pipe with distances and apertures, slices line, and then installs collimators

In [ ]:
L3 = line.get_s_position(aper_exit) - line.get_s_position(aper_start)
lengths = [L1, L2, L3]
apertures = [xt.LimitEllipse(a=r1, b=r1, shift_x=-5.3e-3), 
             xt.LimitEllipse(a=r2, b=r2, shift_x=-5.3e-3),
             xt.LimitEllipse(a=r3, b=r3)] # Only horizontal plane for now
plane = 'H'

def slice_line_for_pipe(line, lengths, pipe_aperture_names, strategy = xt.Teapot(5)):
    tt = line.get_table()
    slicing_strategies = []
    pipe_start = pipe_aperture_names[0]
    pipe_end = pipe_aperture_names[1]

    #Assuming there is only one extra active element in the pipe
    for nn, ee in zip(tt.rows[f'{pipe_start}>>1':pipe_end].name, tt.rows[f'{pipe_start}>>1':pipe_end].element_type):
        if ee.startswith('Drift') or ee.startswith('Marker'):
            continue
        elif ee.startswith('Limit'): #Assuming each element has an aperture at its start
            next_aper = nn
            active_ele = tt.rows[f'{next_aper}>>1'].name[0]
            break
        else:
            raise ValueError(f'Unexpected element type {ee} for element {nn}')
    
    #Find if any element is in the pipe we are trying to define (if not, no need to slice)

In [31]:
tt.rows['vcak.13501.*']

Table: 2 rows, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
vcak.13501.a_aper       1117.77 LimitEllipse   False     False None               False
vcak.13501.b_aper       1118.63 LimitEllipse   False     False None               False

In [ ]:
for nn in tt.rows[(tt.s > 1117.77 - 1e-3 ) & (tt.s <= 1118.63 + 2e-3)].name:
    

Table: 10 rows, 11 cols
name                             s element_type isthick isreplica parent_name iscollective ...
vebb.13490.b_aper          1117.77 LimitRect      False     False None               False
drift_299..1               1117.77 DriftSlice      True     False drift_299          False
vcak.13501.a_aper          1117.77 LimitEllipse   False     False None               False
drift_299..2               1117.77 DriftSlice      True     False drift_299          False
lod.13502_aper_enter       1117.88 LimitEllipse   False     False None               False
lod.13502                  1117.88 Octupole        True     False None               False
lod.13502_aper_exit        1118.55 LimitEllipse   False     False None               False
drift_300..0               1118.55 DriftSlice      True     False drift_300          False
vcak.13501.b_aper          1118.63 LimitEllipse   False     False None               False
drift_300..1               1118.63 DriftSlice      True     Fa

In [38]:
tt.rows['vcak.13501.b_aper'].s[0]

np.float64(1118.6319999999998)

In [70]:
# Look on position to see if aperture exists
pipe_name = 'vcak.13501'
start_pos = 1117.7732
end_pos = 1118.63
start_aper = None
end_aper = None
for nn in tt.rows[(tt.s > start_pos - 1e-3 ) & (tt.s < start_pos + 1e-2 )].name:
    if line[nn].__class__.__name__.startswith('Limit'):
        start_aper = nn

for nn in tt.rows[(tt.s > end_pos - 1e-3 ) & (tt.s < end_pos + 1e-2 )].name:
    if line[nn].__class__.__name__.startswith('Limit'):
        end_aper = nn
        break
if start_aper is None or end_aper is None:
    start_aper = f'{pipe_name}.a_aper'
    end_aper = f'{pipe_name}.b_aper'
    print('To be inserted in line at right positions')

In [71]:
if 'vcak.13501.a_aper' in tt.name:
    print('ok')

ok


In [82]:
def slice_line_for_pipe(line, apertures, pipe_name, pipe_positions=None):
    tt = line.get_table()

    # Placing start and end aperture of the pipe if not already present
    if pipe_positions is None:
        if f'{pipe_name}.a_aper' in tt.name and f'{pipe_name}.b_aper' in tt.name:
            pipe_start = f'{pipe_name}.a_aper'
            pipe_end = f'{pipe_name}.b_aper'
        else:
            raise ValueError(f'Pipe apertures not found in line for {pipe_name}, please provide pipe_positions')
    else:
        pipe_start = None
        pipe_end = None
        for nn in tt.rows[(tt.s > pipe_positions[0] - 1e-3 ) & (tt.s < pipe_positions[0] + 1e-2 )].name:
            if line[nn].__class__.__name__.startswith('Limit'):
                pipe_start = nn

        for nn in tt.rows[(tt.s > pipe_positions[-1] - 1e-3 ) & (tt.s < pipe_positions[-1] + 1e-2 )].name:
            if line[nn].__class__.__name__.startswith('Limit'):
                pipe_end = nn
                break
        if pipe_start is None or pipe_end is None:
            pipe_start = f'{pipe_name}.a_aper'
            pipe_end = f'{pipe_name}.b_aper'
            line.env.elements[pipe_start] = apertures[0].copy()
            line.env.elements[pipe_end] = apertures[-1].copy()
            line.insert([line.env.place(pipe_start, at=pipe_positions[0]),
                         line.env.place(pipe_end, at=pipe_positions[-1])], s_tol=1e-6)
            
    
    # Finding active elements in the pipe
    tt = line.get_table() #update table to find newly placed apertures
    active_elements = []
    for nn, ee in zip(tt.rows[f'{pipe_start}>>1':pipe_end].name, tt.rows[f'{pipe_start}>>1':pipe_end].element_type):
        if ee.startswith('Drift') or ee.startswith('Marker') or ee.startswith('Limit'):
            continue
        else:
            active_elements.append(nn)
    if len(active_elements) == 0:
        print(f'No active elements in pipe {pipe_name}, no need to slice')
        return None
    else:
        print(f'Active elements in pipe {pipe_name}: {active_elements}')

    # Slicing strategies
    slicing_strategies = [xt.Strategy(slicing=None)]
    for nn in active_elements:
        slicing_strategies.append(xt.Strategy(slicing=xt.Teapot(5), name=nn))
    line.slice_thick_elements(slicing_strategies)

In [83]:
apertures = [xt.LimitEllipse(a=r1, b=r1, shift_x=-5.3e-3), 
             xt.LimitEllipse(a=r2, b=r2, shift_x=-5.3e-3),
             xt.LimitEllipse(a=r3, b=r3)] # Only horizontal plane for now
slice_line_for_pipe(line, apertures, 'vcak.13501')

Active elements in pipe vcak.13501: ['lod.13502']


Slicing line:   0%|          | 0/32258 [00:00<?, ?it/s]

In [79]:
aper_start

'vcak.13501.a_aper'

In [84]:
tt = line.get_table()

In [85]:
tt.rows['vcak.13501.a_aper':'vcak.13501.b_aper']

Table: 21 rows, 11 cols
name                             s element_type           isthick isreplica parent_name ...
vcak.13501.a_aper          1117.77 LimitEllipse             False     False None       
drift_299..2               1117.77 DriftSlice                True     False drift_299  
lod.13502_aper_enter       1117.88 LimitEllipse             False     False None       
lod.13502_entry            1117.88 Marker                   False     False None       
lod.13502..entry_map       1117.88 ThinSliceOctupoleEntry   False     False lod.13502  
drift_lod.13502..0         1117.88 DriftSliceOctupole        True     False lod.13502  
lod.13502..0               1117.93 ThinSliceOctupole        False     False lod.13502  
drift_lod.13502..1         1117.93 DriftSliceOctupole        True     False lod.13502  
lod.13502..1               1118.07 ThinSliceOctupole        False     False lod.13502  
drift_lod.13502..2         1118.07 DriftSliceOctupole        True     False lod.13502  
lod.

In [94]:
for nn in changed:
    flange_number = nn-9
    print(flange_number)
    tt.rows[f'vcak.{flange_number}.a_aper':f'vcak.{flange_number}.b_aper'].show()
    print(' ')

10101
name                          s element_type isthick isreplica parent_name iscollective ...
vcak.10101.a_aper       29.8516 LimitEllipse   False     False None               False
drift_4..2              29.8516 DriftSlice      True     False drift_4            False
vcak.10101.b_aper       30.1487 LimitEllipse   False     False None               False
 
11101
name                          s element_type isthick isreplica parent_name iscollective ...
vcak.11101.a_aper       349.828 LimitEllipse   False     False None               False
drift_85..2             349.828 DriftSlice      True     False drift_85           False
vcak.11101.b_aper       350.687 LimitEllipse   False     False None               False
 
11301
name                          s element_type isthick isreplica parent_name iscollective ...
vcak.11301.a_aper       413.824 LimitEllipse   False     False None               False
drift_101..2            413.824 DriftSlice      True     False drift_101          Fals

In [92]:
tt.rows[f'vcak.{flange_number}.a_aper':f'vcak.{flange_number}.b_aper'].show()

name                             s element_type isthick isreplica parent_name iscollective ...
vcak.63501.a_aper          6874.23 LimitEllipse   False     False None               False
drift_1892..2              6874.23 DriftSlice      True     False drift_1892         False
lod.63502_aper_enter       6874.33 LimitEllipse   False     False None               False
lod.63502                  6874.33 Octupole        True     False None               False
lod.63502_aper_exit        6875.01 LimitEllipse   False     False None               False
drift_1893..0              6875.01 DriftSlice      True     False drift_1893         False
vcak.63501.b_aper          6875.09 LimitEllipse   False     False None               False


In [96]:
line['lod.23502'].length

np.float64(0.677)

In [97]:
line['lod.13502']

View of Octupole(k3=0, k3s=0, length=0.677, _order=np.int64(5), inv_factorial_order=0.00833, knl=array([0., 0., 0., 0., 0., 0.]), ksl=array([0., 0., 0., 0., 0., 0.]), edge_entry_active=np.uint64(0), edge_exit_active=np.uint64(0), num_multipole_kicks=np.int64(0), _model=np.int64(0), _integrator=np.int64(0), radiation_flag=np.int64(0), delta_taper=0, _sin_rot_s=-999, _cos_rot_s=-999, _shift_x=0, _shift_y=0, _shift_s=0, _rot_x_rad=0, _rot_y_rad=0, _rot_s_rad_no_frame=0, rot_shift_anchor=0, _internal_record_id=RecordIdentifier(buffer_id=np.int64(0), offset=np.int64(0)))